In [1]:
import numpy as np
from scipy import optimize
from cosmoTransitions import generic_potential as gp
from cosmoTransitions import pathDeformation as pd

In [2]:
GF = 1.16637e-05
v = 1/(np.sqrt(2*np.sqrt(2)*GF))
mHSM = 125.13

In [3]:
def muH2(mS, sintheta):
    """\mu_H square"""
    return 0.5*(mHSM**2 * (1-sintheta**2) + mS**2 * sintheta**2)

def muS2(mS, sintheta):
    """\mu_S square"""
    return sintheta**2 * mHSM**2 + (1 - sintheta**2) * mS**2

def A(mS, sintheta):
    """A parameter"""
    nominator = (mHSM**2 - mS**2) * sintheta * np.sqrt(1-sintheta**2)
    denominator = np.sqrt(2) * v
    return nominator/denominator

def lm(mS, sintheta):
    """\lambda parameter"""
    nominator = (1 - sintheta**2)*mHSM**2 + sintheta**2 * mS**2
    return nominator/(4*v**2)

In [152]:
class model(gp.generic_potential):
    def init(self, mS, sintheta):
        self.Ndim = 2
        self.Tmax = 100
        self.mS = mS
        self.sintheta = sintheta
        self.lm = lm(self.mS, self.sintheta)
        self.A = A(self.mS,self.sintheta)
        self.muH2 = muH2(self.mS,self.sintheta)
        self.muS2 = muS2(self.mS,self.sintheta)
        self.g = 0.65
        self.gY = 0.36
        self.yt = 0.9945
        self.D = (3*self.g**2 + self.gY**2 + 4*self.yt**2)/16.
        self.E = (2*self.g**3+(self.g**2 + self.gY**2)**(3/2))/(48*np.pi)
        self.cs = 1./3
        self.Deff = self.D - self.cs * self.A**2/(4.*self.muS2)
        self.lmeff = self.lm - self.A**2/(2*self.muS2)
        self.T0 = np.sqrt(0.5*self.muH2 - v**2 * self.A**2 /(2*self.muS2))/np.sqrt(self.D - self.cs*self.A**2/(4*self.muS2))
        self.Tc = self.T0*np.sqrt((self.Deff * self.lmeff)/(-self.E**2 + self.Deff*self.lmeff))
        self.strength = 2*self.E/self.lmeff
        self.Tn = False
        self.non_restore_trigger = self.Deff * self.lmeff - self.E**2


    def Vtot(self, X, T, include_radiation=True):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T*T)+1e-100
        phi1 = X[...,0]
        phi2 = X[...,1]
        y = self.D * T2 * phi1**2 - 0.5 * self.muH2 * phi1**2
        y += - self.E * T * phi1**3
        y += 0.25 * self.lm * phi1**4
        y += 0.5*self.muS2*phi2**2 - 0.5 * self.A * (phi1**2 + self.cs * T2 - 2 * v**2)*phi2

        return y

    def truevev(self,T):
        assert T < self.Tc
        nominator = 3.* T * self.E + np.sqrt(9.*self.E**2 * T**2 + 8.*self.Deff * (self.T0**2 - T**2)*self.lmeff)
        denominator = 2.*self.lmeff
        return nominator/denominator

    def Spath(self, X, T):
        X = np.asanyarray(X)
        T = np.asanyarray(T)
        phi1 = X[...,0]
        T2 = (T*T) + 1e-100
        return 0.5*self.A*(phi1**2 + self.cs * T2 - 2 * v**2)/self.muS2

    def tunneling_at_T(self, T):
        assert T < self.Tc
        def V_(x, T=T, V=self.Vtot):
            return V(x,T)
        def dV_(x, T=T, dV=self.gradV):
            return dV(x,T)
        # tobj = pd.fullTunneling([self.findMinimum(T=T),[1e-10,self.Spath([1e-10],T)]],V_,dV_)
        tobj = pd.fullTunneling([[self.truevev(T=T),self.Spath([self.truevev(T=T)],T)],[1e-10,self.Spath([1e-10],T)]],V_,dV_)
        return tobj

    def findTn(self):
        if self.strength >= 12:
            Tmax = 0.95*self.Tc
        elif self.strength >= 4:
            Tmax = 0.99*self.Tc
        else:
            Tmax = self.Tc
        print(Tmax)
        eps = 0.005
        def nuclea_trigger(Tv):
            ST = self.tunneling_at_T(T=Tv).action/Tv
            return ST - 140.
        for i in range(1,1000):
            if nuclea_trigger(Tmax - i*eps) <= 0.:
                break
        Tmin = Tmax - (i-1)*eps
        print(Tmin)
        self.Tn = optimize.brentq(nuclea_trigger,Tmin-1e-10, Tmin-eps,disp=False)

    def strength_Tn(self):
        if not self.Tn:
            self.findTn()
        Tnuc = self.Tn
        return self.truevev(T=Tnuc)/Tnuc

    def beta_over_H_at_Tn(self):
        "Ridders algorithm"
        if not self.Tn:
            self.findTn()
        Tnuc = self.Tn
        if self.Tc-Tnuc >=0.002: eps = 0.001
        else: eps=0.99*(self.Tc-Tnuc)/2
        def SoverT(Tv):
            ST = self.tunneling_at_T(T=Tv).action/Tv
            return ST
        dev = (SoverT(Tnuc-2.*eps) - 8.*SoverT(Tnuc-eps) + 8.*SoverT(Tnuc+eps)- SoverT(Tnuc+2.*eps))/(12.*eps)
        return dev*Tnuc

    def alpha(self):
        if not self.Tn:
            self.findTn()
        Tnuc = self.Tn
        if self.Tc-Tnuc >=0.002: eps = 0.001
        else: eps=0.99*(self.Tc-Tnuc)/2
        def deltaV(T):
            falsev=[0,self.Spath([0],T)]
            truev=self.findMinimum(T=T)
            return self.Vtot(falsev,T)-self.Vtot(truev,T)
        dev = (deltaV(Tnuc-2*eps) - 8.*deltaV(Tnuc-eps) + 8.*deltaV(Tnuc+eps) - deltaV(Tnuc+2.*eps))/(12.*eps) # derivative of deltaV w.r.t T at Tn
        latent=deltaV(Tnuc) - 0.25*Tnuc*dev
        rho_crit = np.pi**2*106.75*Tnuc**4/30.
        return latent/rho_crit

In [153]:
m=model(12,0.02)

In [149]:
m.Tc

106.09807875787067

In [154]:
m.findTn()

106.09807875787067
Path deformation converged. 10 steps. fRatio = 2.25868e-03
Path deformation converged. 1 steps. fRatio = 1.11427e-03
106.09807875787067
Path deformation converged. 9 steps. fRatio = 1.54226e-03
Path deformation converged. 1 steps. fRatio = 1.81702e-03


/Users/isaac/opt/anaconda3/lib/python3.9/site-packages/cosmoTransitions/tunneling1D.py:415: RuntimeWarning: overflow encountered in double_scalars
  r *= 10
/Users/isaac/opt/anaconda3/lib/python3.9/site-packages/cosmoTransitions/tunneling1D.py:415: RuntimeWarning: overflow encountered in double_scalars
  r *= 10


Path deformation converged. 10 steps. fRatio = 2.25868e-03
Path deformation converged. 1 steps. fRatio = 1.11427e-03
Path deformation converged. 10 steps. fRatio = 2.23217e-03
Path deformation converged. 1 steps. fRatio = 1.12056e-03
Path deformation converged. 13 steps. fRatio = 1.04448e-02
Path deformation converged. 1 steps. fRatio = 4.63569e-03
Path deformation converged. 9 steps. fRatio = 1.73306e-02
Path deformation converged. 1 steps. fRatio = 2.52908e-03
Path deformation converged. 12 steps. fRatio = 1.58565e-02
Path deformation converged. 1 steps. fRatio = 9.62282e-03
Path deformation converged. 11 steps. fRatio = 1.85191e-02
Path deformation converged. 1 steps. fRatio = 1.38763e-02
Path deformation converged. 12 steps. fRatio = 1.29065e-02
Path deformation converged. 1 steps. fRatio = 8.03604e-03
Path deformation converged. 11 steps. fRatio = 1.97955e-02
Path deformation converged. 1 steps. fRatio = 1.47496e-02
Path deformation converged. 11 steps. fRatio = 1.97721e-02
Path d

In [151]:
m.strength_Tn()

0.10604169746858608